In [119]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop
import numpy as np
import random
import sys

In [120]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [121]:
# Step 1 - get your data

# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path ='hamlet.txt'

In [122]:
text = open(path).read().lower()  # read the file and convert to lowercase

In [123]:
len(text)

173945

In [124]:
chars = sorted(list(set(text)))

In [125]:
# what position does each character exist at in the prev list
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [126]:
# cut your text into overlapping sequences

maxlen = 40
sentences= []
next_chars=[]

for i in range(0, len(text)-maxlen, 3):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

In [127]:
# vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [128]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]]=1

In [129]:
# build LSTM Model

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [131]:
# train your model
model.fit(x, y, batch_size=128, epochs=1)

Epoch 1/1
57969/57969 [==============================] - 102s - loss: 2.2651    

In [ ]:
# model.save('model.h5')  # creates a HDF5 file 'model.h5'

In [132]:
model.save_weights('model.hdf5')
with open('model.json', 'w') as f:
    f.write(model.to_json())